In [20]:
import pandas as pd
import numpy as np
import string
import re
from tqdm import tqdm_notebook
from scipy.sparse import hstack
from scipy.sparse.csr import csr_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [78]:
df = pd.read_csv('https://raw.githubusercontent.com/esolovev/ling2019/master/module2/twi_data.csv', sep=';')

In [3]:
df.head()

,target,date,text
0,4,Tue Jun 02 02:59:24 PDT 2009,@JackAllTimeLow hope it went good! i couldnt m...
1,0,Sat Jun 06 00:25:20 PDT 2009,@SDI8732 Idk how to do it!!!
2,0,Fri Jun 05 12:07:23 PDT 2009,"@kmwindmill is here ! woop woop , would be bet..."
3,4,Mon Jun 01 14:55:06 PDT 2009,@Daydreamer1984 He explains the tailer better
4,0,Sat Jun 20 15:39:44 PDT 2009,still trying to get a pic on this twitter thin...


Сначала предобработаем тексты, а затем уже разобьем на трейн/тест

Убираю пунктуацию, упоминания других пользователей, начинающихся с @, ссылки и эмодзи (их юникодовские коды)

In [77]:
punct = string.punctuation + '«»—…“”*№–'

def preprocessing(text):
  emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" 
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF"  
                           u"\U0001F1E0-\U0001F1FF"  
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
  clean_text = emoji_pattern.sub(r'', text)
  clean_text = re.sub(r'http\S+', '', clean_text)
  clean_text = re.sub(r'@.*? ', '', clean_text)
  clean_text = ' '.join([word.strip(punct) for word in clean_text.lower().split()])
  return clean_text

In [79]:
df['text'] = df['text'].apply(preprocessing)
df.head()

,target,date,text
0,4,Tue Jun 02 02:59:24 PDT 2009,hope it went good i couldnt make it so i think...
1,0,Sat Jun 06 00:25:20 PDT 2009,idk how to do it
2,0,Fri Jun 05 12:07:23 PDT 2009,is here woop woop would be better if we didn...
3,4,Mon Jun 01 14:55:06 PDT 2009,he explains the tailer better
4,0,Sat Jun 20 15:39:44 PDT 2009,still trying to get a pic on this twitter thin...


In [80]:
SEED = 222
np.random.seed(SEED)
df_train, df_test = train_test_split(df, train_size=0.2, test_size=0.1, stratify=df.target, random_state=SEED)

In [81]:
y_train = df_train.target
y_test = df_test.target

Я буду совмещать бейзлайн 3 (Tfidf по 1-3 граммам слов + Tfidf по 3-4граммам символов) c готовыми эмбеддингами, и на всем этом обучу логистическую регрессию

In [82]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 4))
X_train_tfidf = tfidf_vectorizer.fit_transform(df_train.text)
X_test_tfidf = tfidf_vectorizer.transform(df_test.text)

tfidf_vectorizer_char = TfidfVectorizer(ngram_range=(3, 4), analyzer='char')
X_train_tfidf_char = tfidf_vectorizer_char.fit_transform(df_train.text)
X_test_tfidf_char = tfidf_vectorizer_char.transform(df_test.text)

Использую эмбеддинги GloVe, обученные на твиттере со следующими характеристиками: 2B tweets, 27B tokens, 1.2M vocab, uncased, 200d vectors

In [110]:
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip

--2020-06-24 20:18:27--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2020-06-24 20:18:27--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2020-06-24 20:18:28--  http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [appli

In [113]:
!unzip glove.twitter.27B.zip

Archive:  glove.twitter.27B.zip
  inflating: glove.twitter.27B.25d.txt  
  inflating: glove.twitter.27B.50d.txt  
  inflating: glove.twitter.27B.100d.txt  
  inflating: glove.twitter.27B.200d.txt  


In [9]:
f = open('glove.twitter.27B.200d.txt')

embeddings_index = dict() 
for line in tqdm_notebook(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


Преобразовываем каждое слово в эмбеддинг. Если такого слова нет, то пропускаем. Далее сумму эмбеддингов слов нормируем, чтобы получился вектор предложения (этот кусочек подсмотрела вот здесь https://www.kaggle.com/eswarbabu88/toxic-comment-glove-logistic-regression)

In [47]:
def vectorize_sent(sent):
    Matrix = []
    for w in sent.split():
        try:
            Matrix.append(embeddings_index[w])
        except:
            continue
    Matrix = np.array(Matrix)
    v = Matrix.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(200)
    return v / np.sqrt((v ** 2).sum())

In [83]:
X_train_emb = csr_matrix([vectorize_sent(x) for x in tqdm_notebook(df_train['text'])])
X_test_emb = csr_matrix([vectorize_sent(x) for x in tqdm_notebook(df_test['text'])])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [84]:
X_train = hstack((X_train_tfidf, X_train_tfidf_char, X_train_emb))
X_test = hstack((X_test_tfidf, X_test_tfidf_char, X_test_emb))

In [93]:
model = LogisticRegression(random_state=SEED, solver='liblinear')
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=222, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [94]:
y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_pred, y_test)}')

Accuracy: 0.768
